In [1]:
import sys


sys.path.append("../src/")

In [2]:
from sennet.custom_modules.metrics.surface_dice_metric_fast import create_table_neighbour_code_to_surface_area
import torch
import numpy as np

In [3]:
device = "cpu"


unfold = torch.nn.Unfold(kernel_size=(2, 2), padding=1)


area = create_table_neighbour_code_to_surface_area((1, 1, 1))[1:]
area = torch.from_numpy(area).to(device)  # torch.float32
area.shape

torch.Size([255])

In [4]:
bases = torch.tensor([
    [c=="1" for c in f"{i:08b}"]
    for i in range(1, 256)
]).float().permute((1, 0)).to(device)
print(f"{bases.shape=}")
print(bases)

bases.shape=torch.Size([8, 255])
tensor([[0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 1., 1.,  ..., 0., 1., 1.],
        [1., 0., 1.,  ..., 1., 0., 1.]])


In [5]:
batch_size = 3
zs = 2
ys = 10
xs = 10

pred = torch.rand((batch_size, zs, ys, xs)).to(device)
labels = (torch.rand((batch_size, zs, ys, xs)) > 0.5).to(device)

In [6]:
unfolded_pred = unfold(pred)
unfolded_labels = unfold(labels.float())

permuted_unfolded_pred = unfolded_pred.permute((0, 2, 1)).unsqueeze(-1)
permuted_unfolded_labels = unfolded_labels.permute((0, 2, 1)).unsqueeze(-1)

print(f"{unfolded_pred.shape=}")
print(f"{unfolded_labels.shape=}")
print(f"{permuted_unfolded_pred.shape=}")
print(f"{permuted_unfolded_labels.shape=}")

unfolded_pred.shape=torch.Size([3, 8, 121])
unfolded_labels.shape=torch.Size([3, 8, 121])
permuted_unfolded_pred.shape=torch.Size([3, 121, 8, 1])
permuted_unfolded_labels.shape=torch.Size([3, 121, 8, 1])


In [7]:
a_mat = bases.tile((3, 121, 1, 1))
b_mat = permuted_unfolded_pred
bases_weights = torch.linalg.lstsq(a_mat, b_mat).solution

In [8]:
print(bases_weights.shape)
print(area.shape)

torch.Size([3, 121, 255, 1])
torch.Size([255])


In [22]:
pred_areas = (area[None, None, :, None] * bases_weights).squeeze(-1).sum(2)
print(pred_areas.shape)

torch.Size([3, 121])


In [10]:
label_cubes_byte = torch.zeros((unfolded_labels.shape[0], unfolded_labels.shape[2]), dtype=torch.int32, device=device)
for k in range(8):
    label_cubes_byte += unfolded_labels[:, k, :].to(torch.int32) << k
label_cubes_byte.shape

torch.Size([3, 121])

In [21]:
label_areas = torch.zeros((label_cubes_byte.shape[0], label_cubes_byte.shape[1]), dtype=torch.float32, device=device)
for b in range(label_cubes_byte.shape[0]):
    label_areas[b, :] = torch.where(label_cubes_byte[b, :] == 0, label_areas[b, :], area[label_cubes_byte[b, :] - 1])
label_areas.shape

torch.Size([3, 121])

In [15]:
smooth = 1e-3
intersection = ...
numerator = 2 * intersection
denominator = label_areas.sum() + pred_areas.sum()
dice = 1 - ((numerator + smooth) / (denominator + smooth))

TypeError: scatter() received an invalid combination of arguments - got (Tensor, int, Tensor), but expected one of:
 * (Tensor input, int dim, Tensor index, Tensor src, *, Tensor out)
 * (Tensor input, int dim, Tensor index, Tensor src, *, str reduce, Tensor out)
 * (Tensor input, name dim, Tensor index, Tensor src)
 * (Tensor input, int dim, Tensor index, Number value, *, Tensor out)
 * (Tensor input, int dim, Tensor index, Number value, *, str reduce, Tensor out)
 * (Tensor input, name dim, Tensor index, Number value)


In [ ]:
unfolded_labels.shape

In [ ]:
smooth = 1e-3
numerator = 
denominator = pred_areas.sum() + 

In [ ]:
pred_areas.shape

In [ ]:
torch.abs(torch.sum(permuted_unfolded_pred))

In [ ]:
torch.abs(torch.sum(a_mat))

In [ ]:
a = bases
# b = permuted_unfolded_pred[0, 0]
b = a[:, 0]
print(f"{a.shape = }")
print(f"{b.shape = }")
print(a)
print(b)
sol = torch.linalg.lstsq(bases, b)
print(sol.rank)
print(sol.solution.reshape(-1))

In [ ]:
torch.linalg.matrix_rank(a)

In [ ]:
a_np = a.cpu().numpy()
b_np = b.cpu().numpy()

weights, residuals, rank, singular_vals = np.linalg.lstsq(a_np, b_np, rcond=None)

print(a_np.shape)
print(b_np.shape)
print(b_np)
print(np.sum(np.abs(weights)))
print(residuals)
print(rank)
print(singular_vals)

In [ ]:
for i in range(256):
    print(a_np[:, i])